# Extracting Morphological Signatures

In this notebook, we extract morphological signatures associated with two distinct cellular states:  
- **On-morphology features**: Features that significantly change with the cellular state  
- **Off-morphology features**: Features that do not show significant changes

We identify and categorize features as either on- or off-morphology signatures using a systematic workflow.  
This approach is applied to three datasets: Pilot-CFReT, MitoCheck, and CPJUMP1 (CRISPR only).

In [1]:
import sys
import json
import pathlib
import pprint
from collections import Counter

import polars as pl

sys.path.append("../../")
from utils.signatures import get_signatures
from utils.io_utils import load_profiles
from utils.data_utils import split_meta_and_features

In [2]:
# parameters
method = "ks_test"

Setting input and output paths

In [3]:
# setting data directory path
data_dir = pathlib.Path("../0.download-data/data/").resolve(strict=True)
data_sc_profiles_path = (data_dir / "sc-profiles").resolve(strict=True)
cpjump1_profiles_dir_path = (data_sc_profiles_path / "cpjump1").resolve(strict=True)
data_results_dir = pathlib.Path("../0.download-data/results/").resolve(strict=True)

# setting dataset paths
cpjump1_negcon_profile_path =  (cpjump1_profiles_dir_path / "negcon" ).resolve(strict=True).glob("*.parquet")
cpjump1_poscon_profile_path =  (cpjump1_profiles_dir_path / "poscon" / "poscon_cp_df.parquet").resolve(strict=True)
cpjump1_trt_crispr_profile_path = (
    cpjump1_profiles_dir_path / "trt-profiles" / "cpjump1_crispr_trt_profiles.parquet"
).resolve(strict=True)

# setting mitocheck profile path
mitocheck_profiles_path = (
    data_sc_profiles_path
    / "mitocheck"
    / "concat_mitocheck_cp_profiles_shared_feats.parquet"
).resolve(strict=True)

# setting cfret profile path
cfret_plate_plat = (
    data_sc_profiles_path
    / "cfret"
    / "localhost230405150001_sc_feature_selected.parquet"
).resolve(strict=True)

# setting making a results directory and creating it
results_dir = pathlib.Path("results").resolve()
results_dir.mkdir(exist_ok=True)

# now making a "signature_results" within the results directory
signature_results_dir = (results_dir / "signature_results").resolve()
signature_results_dir.mkdir(exist_ok=True)


## Loading profiles

### Loading CPJUMP CRISPR profiles

In [4]:
# loading all the cpjump1 poscon_cp controls 
negcon_dfs = [load_profiles(df_path) for df_path in cpjump1_negcon_profile_path]

# loading positive 
poscon_df = load_profiles(cpjump1_poscon_profile_path)

In [5]:
poscon_df

Metadata_gene
str
"""EZH2"""
"""EZH2"""
"""EZH2"""
"""EZH2"""
"""EZH2"""
…
"""DYRK1B"""
"""DYRK1B"""
"""DYRK1B"""


## Generating on and off morpholgy signatures 

### Get signatures from CPJUMP1 dataset 

In [ ]:
# generating metadata dataframe
cp_jump_meta_df = sc_jump_crispr_profiles[sc_jump_crispr_meta]

# creating positive control genes list
negcon_profiles_df = sc_jump_crispr_profiles.filter(
    pl.col("Metadata_control_type") == "negcon"
)

# treatment profiles
trt_profiles_df = sc_jump_crispr_profiles.filter(pl.col("Metadata_pert_type") == "trt")

# selecting positive control profiles
# poscon_cp = known chemical probs that module specific genes
poscon_profiles_df = sc_jump_crispr_profiles.filter(
    pl.col("Metadata_control_type") == "poscon_cp"
)
poscon_genes = (
    cp_jump_meta_df.filter(pl.col("Metadata_control_type") == "poscon_cp")[
        "Metadata_gene"
    ]
    .unique()
    .sort()
    .to_list()
)

# displaying the number of positive control genes and their names
pprint.pprint(f"Number of positive control genes: {len(poscon_genes)}")
pprint.pprint(f"These are poscon genes: {poscon_genes}")

# display dataframe of positive control profiles
poscon_profiles_df.head()

'Number of positive control genes: 13'
("These are poscon genes: ['AURKB', 'BRD4', 'CLK1', 'DYRK1B', 'ERBB2', 'EZH2', "
 "'FLT3', 'HDAC3', 'IGF1R', 'JAK1', 'MET', 'PAK1', 'USP1']")


index,Metadata_broad_sample,Metadata_ImageNumber,Metadata_Plate,Metadata_Site,Metadata_Well,Metadata_TableNumber,Metadata_ObjectNumber_cytoplasm,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ObjectNumber_cells,Metadata_ObjectNumber,Metadata_gene,Metadata_pert_type,Metadata_control_type,Metadata_target_sequence,Metadata_negcon_control_type,__index_level_0__,Nuclei_Texture_InverseDifferenceMoment_ER_5_01_256,Cytoplasm_AreaShape_Zernike_4_2,Cytoplasm_AreaShape_Zernike_9_3,Nuclei_RadialDistribution_RadialCV_AGP_2of4,Nuclei_Correlation_Correlation_DNA_HighZBF,Cells_Texture_Correlation_HighZBF_5_00_256,Cells_AreaShape_Solidity,Nuclei_RadialDistribution_MeanFrac_HighZBF_4of4,Nuclei_AreaShape_Orientation,Nuclei_Texture_Correlation_ER_5_02_256,Cytoplasm_Correlation_Correlation_DNA_LowZBF,Cytoplasm_Texture_InfoMeas2_DNA_10_01_256,Cells_RadialDistribution_FracAtD_DNA_2of4,Cells_RadialDistribution_MeanFrac_HighZBF_1of4,Nuclei_RadialDistribution_MeanFrac_Mito_4of4,Cells_Correlation_RWC_DNA_ER,Cells_Texture_InfoMeas2_ER_3_00_256,Cells_RadialDistribution_MeanFrac_HighZBF_2of4,Cytoplasm_RadialDistribution_RadialCV_ER_3of4,…,Nuclei_AreaShape_MinFeretDiameter,Nuclei_AreaShape_Zernike_5_3,Nuclei_AreaShape_Zernike_4_2,Nuclei_RadialDistribution_MeanFrac_HighZBF_3of4,Cytoplasm_Granularity_1_Brightfield,Nuclei_Correlation_Correlation_ER_Mito,Nuclei_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Solidity,Nuclei_RadialDistribution_FracAtD_DNA_3of4,Nuclei_AreaShape_Zernike_8_4,Nuclei_Intensity_MassDisplacement_HighZBF,Cytoplasm_Texture_Correlation_LowZBF_3_03_256,Cells_RadialDistribution_RadialCV_Mito_1of4,Nuclei_Intensity_MassDisplacement_AGP,Nuclei_Correlation_Correlation_AGP_Mito,Nuclei_RadialDistribution_MeanFrac_HighZBF_1of4,Cells_RadialDistribution_RadialCV_DNA_2of4,Nuclei_RadialDistribution_RadialCV_RNA_3of4,Cells_AreaShape_Zernike_7_3,Nuclei_RadialDistribution_MeanFrac_ER_1of4,Cytoplasm_Correlation_Overlap_ER_RNA,Cells_Texture_Correlation_LowZBF_5_01_256,Cytoplasm_Texture_Correlation_HighZBF_3_01_256,Nuclei_Texture_Correlation_LowZBF_3_02_256,Cells_RadialDistribution_RadialCV_AGP_3of4,Nuclei_RadialDistribution_MeanFrac_AGP_1of4,Cells_RadialDistribution_MeanFrac_Brightfield_1of4,Cytoplasm_Correlation_Correlation_DNA_HighZBF,Nuclei_Intensity_MassDisplacement_DNA,Cytoplasm_RadialDistribution_MeanFrac_DNA_4of4,Cells_Correlation_Correlation_AGP_DNA,Cytoplasm_Texture_InfoMeas2_RNA_3_01_256,Cells_RadialDistribution_RadialCV_DNA_4of4,Nuclei_Correlation_Correlation_DNA_LowZBF,Cytoplasm_Texture_Correlation_LowZBF_5_00_256,Cytoplasm_RadialDistribution_RadialCV_HighZBF_2of4,Nuclei_RadialDistribution_MeanFrac_Brightfield_1of4
u32,str,i64,str,i64,str,str,i64,f64,f64,i64,i64,str,str,str,str,str,i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
44370,"""BRDN0001057455""",469,"""BR00118041""",1,"""C05""","""251377083526098455688025670966…",1,1.0,1.0,1,1,"""EZH2""","""control""","""poscon_cp""","""AGAAGGGACCAGTTTGTTGG""",null,44729,-0.720737,-0.99562,0.608404,-0.075396,-0.754694,1.192771,1.189916,1.121562,1.073525,-0.101923,-2.332029,0.813718,1.891952,-0.528007,0.052382,0.492962,0.542304,-0.829857,2.179971,…,0.857316,1.041054,-0.365782,-1.650042,-0.066899,0.377524,-1.456956,1.227787,1.236125,-1.120438,1.545298,1.002021,0.32726,1.629706,0.583361,0.452936,1.209641,0.538964,0.964841,-0.333102,0.552967,0.430044,0.961234,0.543158,-0.0862,0.547685,-0.48022,-0.793249,-0.273273,-0.807079,0.122084,0.623559,-0.822849,1.389746,0.677316,0.586454,0.952611
44371,"""BRDN0001057455""",469,"""BR00118041""",1,"""C05""","""251377083526098455688025670966…",2,2.0,2.0,2,2,"""EZH2""","""control""","""poscon_cp""","""AGAAGGGACCAGTTTGTTGG""",null,44730,-2.029901,1.087042,0.715875,1.406727,1.743212,1.184291,1.902819,-1.705426,-1.695478,0.551302,-0.896374,1.957329,0.384656,0.513912,-0.6083

In [ ]:
# group by plate first
results = {}
for gene_name, gene_group_df in trt_profiles_df.group_by("Metadata_gene"):
    gene_name = gene_name[0]  # extract string value from tuple

    # getting signatures for the current gene
    on_morph_sig, off_morph_sig = get_signatures(
        ref_profiles=negcon_profiles_df,
        exp_profiles=gene_group_df,
        morph_feats=sc_jump_crispr_feats,
        method=method,
    )
    # Counting compartment signatures
    on_morph_compartments_counts = dict(
        Counter([feat.split("_")[0] for feat in on_morph_sig])
    )

    # store in dict
    results[f"negcon_{gene_name}"] = {
        "on_morph_sig": on_morph_sig,
        "off_morph_sig": off_morph_sig,
        "on_morph_compartments_counts": on_morph_compartments_counts,
    }

# writing results to a json file
with open(
    signature_results_dir / f"{method}_negcon_trt_signature_results.json", "w"
) as f:
    json.dump(results, f, indent=4)


In [ ]:
# create a dataframe from the results
counts = []
for key, value in results.items():
    gene_name = key.replace("negcon_", "")  # remove prefix to get just gene name
    compartment_counts = value["on_morph_compartments_counts"]

    # get counts for each compartment, defaulting to 0 if not present
    nuclei_count = compartment_counts.get("Nuclei", 0)
    cytoplasm_count = compartment_counts.get("Cytoplasm", 0)
    cells_count = compartment_counts.get("Cells", 0)

    counts.append([gene_name, nuclei_count, cytoplasm_count, cells_count])

# creating a dataframe with the counts
cols = ["Gene", "Nuclei", "Cytoplasm", "Cells"]
on_sig_compartment_counts = pl.DataFrame(counts, schema=cols)

# save
on_sig_compartment_counts.write_csv(
    signature_results_dir / f"{method}_negcon_trt_on_sig_compartment_counts.csv",
)


### Generating signatures from Mitocheck data


Loading in mitocheck data

In [4]:
# loading MitoCheck profiles
mitocheck_profiles = load_profiles(mitocheck_profiles_path)

# splitting metadata and features columns
mito_meta = mitocheck_profiles.columns[:12]
mito_feats = mitocheck_profiles.drop(mito_meta).columns


Splitting them into control and treated profiles

In [ ]:
# selecting column that contains the phenotype class
phenotype_class = "Mitocheck_Phenotypic_Class"

# splitting control and treated profiles
mitocheck_negcontrol_profiles = mitocheck_profiles.filter(
    pl.col(phenotype_class) == "negcon"
)
mitocheck_trt_profiles = mitocheck_profiles.filter(pl.col(phenotype_class) != "negcon")

In [ ]:
# group by plate first
results = {}
for class_name, class_group_df in mitocheck_trt_profiles.group_by(phenotype_class):
    class_name = class_name[0]  # extract string value from tuple

    # getting signatures for the current class
    on_morph_sig, off_morph_sig = get_signatures(
        ref_profiles=mitocheck_negcontrol_profiles,
        exp_profiles=class_group_df,
        morph_feats=mito_feats,
        method=method,
    )
    # Counting compartment signatures
    off_morph_compartments_counts = dict(
        Counter([feat.split("_")[0] for feat in off_morph_sig])
    )
    on_morph_compartments_counts = dict(
        Counter([feat.split("_")[0] for feat in on_morph_sig])
    )

    # store in dict
    results[f"negcon_{class_name}"] = {
        "off_morph_compartments_counts": off_morph_compartments_counts,
        "on_morph_compartments_counts": on_morph_compartments_counts,
        "on_morph_sig": on_morph_sig,
        "off_morph_sig": off_morph_sig,
    }


# writing results to a json file
with open(
    signature_results_dir / f"{method}_mitocheck_trt_signature_results.json", "w"
) as f:
    json.dump(results, f, indent=4)

### Generating signatures from CFReT data

In [10]:
# loading cfret profiles
phenotype_class = "Metadata_treatment"
cfret_profiles = load_profiles(cfret_plate_plat)

# splitting metadata and features columns
cfret_meta, cfret_feats = split_meta_and_features(cfret_profiles)

# splitting negative control and treatment profiles
negcon_cfret_profiles = cfret_profiles.filter(pl.col("Metadata_treatment") == "DMSO")
trt_cfret_profiles = cfret_profiles.filter(pl.col("Metadata_treatment") != "DMSO")

In [14]:
results = {}
for trt_name, trt_group_df in trt_cfret_profiles.group_by(phenotype_class):
    trt_name = trt_name[0]

    # getting signatures for the current class
    on_morph_sig, off_morph_sig = get_signatures(
        ref_profiles=negcon_cfret_profiles,
        exp_profiles=trt_group_df,
        morph_feats=cfret_feats,
        method=method,
    )
    # counting compartment signatures
    off_morph_compartments_counts = dict(
        Counter([feat.split("_")[0] for feat in off_morph_sig])
    )
    on_morph_compartments_counts = dict(
        Counter([feat.split("_")[0] for feat in on_morph_sig])
    )

    # store in dict
    results[f"negcon_{trt_name}"] = {
        "off_morph_compartments_counts": off_morph_compartments_counts,
        "on_morph_compartments_counts": on_morph_compartments_counts,
        "on_morph_sig": on_morph_sig,
        "off_morph_sig": off_morph_sig,
    }

# writing results to a json file
with open(
    signature_results_dir / f"{method}_cfret_trt_signature_results.json", "w"
) as f:
    json.dump(results, f, indent=4)
